# 시계열 분석 - 인공신경망 모델 (ANN)
## Team 4

이 노트북은 기본 인공신경망(ANN, 여기서는 Multi-Layer Perceptron, MLP)을 사용하여 시계열 예측을 수행하는 과정을 보여줍니다.
ANN/MLP는 입력 특성(여기서는 과거 시점의 값, 날짜 정보 등)과 타겟 변수 간의 비선형 관계를 학습할 수 있는 기본적인 딥러닝 모델입니다.
LSTM과 달리 시간적 순서를 직접 고려하지는 않지만, 적절한 특성 엔지니어링을 통해 시계열 예측에 활용될 수 있습니다.

## 분석 흐름
1. **문제 정의 및 목표 설정**: 예측 대상, 목표, 평가 지표를 정의합니다.
2. **데이터 탐색 및 전처리**: 데이터를 로드하고 패턴을 탐색하며, 결측치 처리, 특성 엔지니어링, 스케일링, 데이터 분할을 수행합니다.
3. **모델 선택 및 구현 (ANN/MLP)**: 다양한 구조의 MLP 모델을 정의하고 컴파일합니다.
4. **모델 학습 및 평가**: 모델을 학습시키고 검증/테스트 데이터로 성능을 평가하며 학습 곡선을 확인합니다.
5. **모델 비교**: 여러 MLP 모델의 성능을 비교하여 최적 모델을 선정합니다.
6. **결과 해석 및 인사이트 도출**: 최적 모델의 예측 결과를 분석하고, 미래 예측 시뮬레이션(단순화된 방식)을 수행하며 인사이트를 도출합니다.

## 1. 문제 정의 및 목표 설정

이 템플릿은 기본적인 MLP 모델을 이용한 시계열 예측의 기본 구조를 제공합니다.

**목표:**
- MLP 모델을 사용하여 시계열 데이터를 예측합니다.
- **윈도우 기반 특성 엔지니어링**: 과거 시점의 값(lag), 날짜 정보, 이동 통계량 등을 특성으로 변환하여 MLP 모델의 입력으로 사용합니다.
- 다양한 MLP 모델 구조(은닉층 수, 뉴런 수, 활성화 함수 등)를 실험하고 비교합니다.
- 간단한 딥러닝 모델로서의 성능 기준점을 확인합니다.

**평가 지표:** RMSE, MAE, R² Score

### 사용자 파라미터 설정
분석 데이터, 모델 구조, 학습 파라미터를 설정합니다.
MLP 모델은 입력으로 사용할 특성을 정의하는 `WINDOW_SIZE` 파라미터가 중요합니다.

In [ ]:
# 데이터 파라미터
DATA_PATH = "../data/raw/your_data.csv"   # 데이터 파일 경로
DATE_COL = "date"                         # 날짜 컬럼명
TARGET_COL = "value"                      # 예측할 타겟 컬럼명
TRAIN_SIZE = 0.7                          # 훈련 데이터 비율
VALIDATION_SIZE = 0.1                     # 검증 데이터 비율 (7:1:2 분할)

# 모델 파라미터
WINDOW_SIZE = 7                           # 입력 윈도우 크기 (과거 며칠의 데이터로 예측할지)
EPOCHS = 100                              # 최대 학습 에폭
BATCH_SIZE = 32                           # 배치 크기
PATIENCE = 20                             # 조기 종료 파라미터
FORECAST_HORIZON = 30                     # 미래 예측 기간 (일 단위 등)
RANDOM_STATE = 42                         # 랜덤 시드 (재현성을 위해 추가)

# 출력 디렉토리
OUTPUT_DIR = '../plots'                   # 시각화 결과 저장 경로
DATA_OUTPUT_DIR = '../data/processed'     # 전처리된 데이터 저장 경로

### 필요한 라이브러리 임포트
데이터 처리, 시각화, 시계열 분석, 딥러닝 모델링(TensorFlow/Keras)에 필요한 라이브러리를 불러옵니다.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
import tensorflow as tf
# Keras를 독립 패키지로 사용
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Input
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
import statsmodels.tsa.api as tsa
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
import os
import warnings
from datetime import datetime, timedelta

# 경고 무시
warnings.filterwarnings('ignore')

# 랜덤 시드 설정 (결과 재현성을 위해)
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)
keras.utils.set_random_seed(RANDOM_STATE)  # keras 시드 설정 추가

# 그래프 스타일 설정
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# 결과 저장 디렉토리 생성 코드 제거
# 프로젝트 지침에 따라 필요 폴더는 이미 존재한다고 가정

# # GPU 메모리 설정 (필요시 주석 해제 및 환경에 맞게 설정)
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#         print("GPU 메모리 증가 설정 완료")
#     except RuntimeError as e:
#         print(f"GPU 설정 오류: {e}")
# else:
#     print("사용 가능한 GPU가 없습니다. CPU를 사용합니다.")

## 2. 데이터 탐색 및 전처리
MLP 모델 학습을 위한 데이터를 준비하는 과정입니다.
데이터 로드, 기본 정보 확인, 결측치 처리, 시각화를 통한 패턴 분석, 특성 생성, 스케일링, 데이터 분할 등을 포함합니다.

### 2.1 데이터 로드 및 기본 탐색
지정된 경로에서 데이터를 로드하고, 날짜 인덱스 설정, 타겟 변수 확인 및 기본 정보를 출력합니다.

In [ ]:
# 데이터 로드 (사용자가 제공한 경로)
df = pd.read_csv(DATA_PATH)
print(f"데이터를 성공적으로 로드했습니다. 크기: {df.shape}")

# 날짜 열 처리 (사용자 지정 컬럼 사용)
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
df.set_index(DATE_COL, inplace=True)
print(f"'{DATE_COL}' 열을 인덱스로 설정했습니다.")

# 타겟 변수 확인 (사용자 지정 컬럼 사용)
if TARGET_COL not in df.columns:
    # TARGET_COL이 없는 경우 오류 발생 또는 경고 후 첫 번째 숫자형 컬럼 사용 등의 처리가 필요하나,
    # 템플릿 단순화를 위해 사용자가 올바른 컬럼명을 제공하는 것을 전제합니다.
    print(f"경고: 지정된 타겟 컬럼 '{TARGET_COL}'이 데이터에 없습니다. 첫 번째 숫자형 컬럼을 사용하거나 코드를 수정하세요.")
    # 또는 raise ValueError(f"Target column '{TARGET_COL}' not found in data.")

print(f"\n타겟 변수: {TARGET_COL}")

# 데이터 요약 정보
print("\n데이터 기본 정보:")
print(df.info())

print("\n기술 통계량:")
print(df.describe())

# 데이터 처음과 끝 확인
print("\n데이터 처음 5행:")
print(df.head())
print("\n데이터 마지막 5행:")
print(df.tail())

### 2.2 시계열 시각화 및 패턴 분석
전처리된 데이터를 시각화하여 패턴을 분석하고, 정상성 여부를 확인합니다.
- **시계열 플롯**: 시간에 따른 데이터 변화 추세를 확인합니다.
- **결측치 처리**: 선형 보간법을 사용하여 결측치를 채웁니다.
- **ACF/PACF 플롯**: 데이터의 자기상관 구조를 파악합니다.
- **ADF 검정**: 시계열의 정상성을 통계적으로 검정합니다.

In [ ]:
# 기본 시계열 시각화
plt.figure(figsize=(14, 7))
plt.plot(df.index, df[TARGET_COL])
plt.title('시계열 데이터')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.grid(True)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/ann_시계열데이터.png')
plt.show()

# 결측치 확인 및 처리
print("\n결측치 개수:")
print(df.isnull().sum())

# 시나리오 A: 결측치가 있는 경우 - 선형 보간법으로 처리
# 시계열의 연속성을 최대한 유지하면서 결측치를 채우는 방법
# 대부분의 시계열 분석에서 권장되는 기본 접근법
df_clean = df.copy()
df_clean = df_clean.interpolate(method='time')
    
# 시작/끝에 결측치가 남아있을 경우
if df_clean.isnull().sum().sum() > 0:
    df_clean = df_clean.fillna(method='bfill').fillna(method='ffill')
    
print(f"결측치 처리 완료. 남은 결측치: {df_clean.isnull().sum().sum()}")

# 시나리오 B: 결측치가 없는 경우 (주석 처리됨)
# df_clean = df.copy()
# print("결측치가 없습니다.")

# 시계열 분석 - ACF, PACF
plt.figure(figsize=(14, 7))
plt.subplot(211)
plot_acf(df_clean[TARGET_COL].dropna(), lags=40, ax=plt.gca())
plt.title('자기상관함수(ACF)')

plt.subplot(212)
plot_pacf(df_clean[TARGET_COL].dropna(), lags=40, ax=plt.gca())
plt.title('편자기상관함수(PACF)')

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/ann_자기상관.png')
plt.show()

# 정상성 검정
adf_result = adfuller(df_clean[TARGET_COL].dropna())
print('ADF 테스트 결과:')
print(f'ADF 통계값: {adf_result[0]:.4f}')
print(f'p-value: {adf_result[1]:.4f}')
print('임계값:')
for key, value in adf_result[4].items():
    print(f'   {key}: {value:.4f}')

is_stationary = adf_result[1] < 0.05
print(f"시계열 정상성 여부: {'정상적일 가능성 높음 (p < 0.05)' if is_stationary else '비정상적일 가능성 높음 (p >= 0.05)'}")

### 2.3 특성 엔지니어링 및 윈도우 생성
시계열 데이터를 MLP 모델의 입력 특성으로 변환하는 과정입니다.
- **특성 생성**: 날짜 기반 특성, 시차(lag) 특성, 이동(rolling) 특성 등을 생성합니다. `WINDOW_SIZE`는 시차 및 이동 특성 생성에 사용되는 과거 기간을 의미합니다.
- **결측치 처리**: 특성 생성 과정에서 발생하는 초기 결측치를 제거합니다.
- **데이터 스케일링**: 생성된 특성(X)과 타겟(y)을 각각 적절한 스케일러(`StandardScaler`, `MinMaxScaler`)로 변환합니다.
- **데이터 분할**: 스케일링된 특성 데이터를 훈련, 검증, 테스트 세트로 분할합니다. 시계열 데이터의 순서를 유지합니다.
**주의**: MLP 모델은 LSTM과 달리 시퀀스 형태의 입력이 필요하지 않으므로, `create_sequences` 함수를 사용하지 않습니다.

In [ ]:
# 시나리오 A: 시간 관련 특성 추가 (DatetimeIndex가 있는 경우)
# 시간 정보에서 다양한 특성 추출하여 모델 성능 향상
if isinstance(df_clean.index, pd.DatetimeIndex):
    print("시간 관련 특성 추가 중...")
    # 원본 데이터 복사
    df_features = df_clean.copy()
    
    # 시간 관련 특성
    df_features['dayofweek'] = df_features.index.dayofweek
    df_features['month'] = df_features.index.month
    df_features['year'] = df_features.index.year
    
    # 시간 특성 사인/코사인 변환 (주기성 포착)
    df_features['dayofweek_sin'] = np.sin(2 * np.pi * df_features.index.dayofweek / 7)
    df_features['dayofweek_cos'] = np.cos(2 * np.pi * df_features.index.dayofweek / 7)
    df_features['month_sin'] = np.sin(2 * np.pi * df_features.index.month / 12)
    df_features['month_cos'] = np.cos(2 * np.pi * df_features.index.month / 12)
    
    # 시차 특성 (이전 시점의 값)
    for i in range(1, WINDOW_SIZE + 1):
        df_features[f'lag_{i}'] = df_features[TARGET_COL].shift(i)
    
    # 이동 평균 및 표준편차
    df_features['rolling_mean_7'] = df_features[TARGET_COL].rolling(window=7).mean()
    df_features['rolling_std_7'] = df_features[TARGET_COL].rolling(window=7).std()
    
    print("생성된 특성:")
    print(df_features.columns.tolist())
    
    # 결측치 처리
    df_features = df_features.dropna()
    print(f"특성 생성 완료. 최종 데이터 크기: {df_features.shape}")
else:
    # 시나리오 B: DatetimeIndex가 없는 경우 (단순 시차 변수만 생성)
    print("DatetimeIndex가 없어 단순 시차 변수만 생성합니다.")
    
    df_features = df_clean.copy()
    for i in range(1, WINDOW_SIZE + 1):
        df_features[f'lag_{i}'] = df_features[TARGET_COL].shift(i)
    
    # 결측치 처리
    df_features = df_features.dropna()
    print(f"특성 생성 완료. 최종 데이터 크기: {df_features.shape}")

# 데이터 스케일링
print("\n데이터 스케일링...")
# 타겟 변수와 특성 분리
X = df_features.drop(columns=[TARGET_COL])
y = df_features[TARGET_COL]

# 특성 스케일링
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

# 타겟 스케일링
scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()

# 데이터 분할
train_size = int(len(X_scaled) * TRAIN_SIZE)
val_size = int(len(X_scaled) * VALIDATION_SIZE)
test_size = len(X_scaled) - train_size - val_size

# 훈련, 검증, 테스트 데이터 분할 (7:1:2 비율)
X_train = X_scaled[:train_size]
y_train = y_scaled[:train_size]

X_val = X_scaled[train_size:train_size+val_size]
y_val = y_scaled[train_size:train_size+val_size]

X_test = X_scaled[train_size+val_size:]
y_test = y_scaled[train_size+val_size:]

print(f"훈련 데이터: X:{X_train.shape}, y:{y_train.shape} (70%)")
print(f"검증 데이터: X:{X_val.shape}, y:{y_val.shape} (10%)")
print(f"테스트 데이터: X:{X_test.shape}, y:{y_test.shape} (20%)")

# 데이터 분할 시각화 (원본 값 기준)
original_dates = df_features.index
train_dates = original_dates[:train_size]
val_dates = original_dates[train_size:train_size+val_size]
test_dates = original_dates[train_size+val_size:]

plt.figure(figsize=(14, 7))
plt.plot(train_dates, df_features.iloc[:train_size][TARGET_COL], label='훈련 데이터 (70%)')
plt.plot(val_dates, df_features.iloc[train_size:train_size+val_size][TARGET_COL], label='검증 데이터 (10%)')
plt.plot(test_dates, df_features.iloc[train_size+val_size:][TARGET_COL], label='테스트 데이터 (20%)')
plt.title('데이터 분할 (7:1:2 비율)')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.legend()
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/ann_데이터분할.png')
plt.show()

## 3. 모델 선택 및 구현
기본적인 MLP 모델 구조를 정의하고 학습 준비를 합니다.

### 3.1 인공신경망 모델 정의
Keras Sequential API를 사용하여 다양한 구조의 MLP 모델을 정의하는 함수들을 만듭니다.
- **Dense 레이어**: 완전 연결 레이어. `units`는 뉴런 수, `activation`은 활성화 함수(예: 'relu')를 지정합니다.
- **BatchNormalization**: 학습 안정화 및 속도 향상.
- **Dropout**: 과적합 방지.
모델 구성(Simple, Deep, Complex)을 리스트로 정의하여 비교 실험을 준비합니다.

In [ ]:
# 여러 ANN 모델 정의
def build_simple_ann(input_dim):
    """
    간단한 단층 신경망 모델
    """
    model = Sequential([
        Dense(32, activation='relu', input_dim=input_dim),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

def build_deep_ann(input_dim):
    """
    다층 신경망 모델
    """
    model = Sequential([
        Dense(64, activation='relu', input_dim=input_dim),
        BatchNormalization(),
        Dense(32, activation='relu'),
        BatchNormalization(),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

def build_complex_ann(input_dim):
    """
    드롭아웃을 포함한 복잡한 신경망 모델
    """
    model = Sequential([
        Dense(128, activation='relu', input_dim=input_dim),
        BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# 다양한 모델 구성
ann_models = [
    {'name': 'SimpleANN', 'build_fn': build_simple_ann},
    {'name': 'DeepANN', 'build_fn': build_deep_ann},
    {'name': 'ComplexANN', 'build_fn': build_complex_ann}
]

# 콜백 함수 설정
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=PATIENCE,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=PATIENCE//2,
        min_lr=0.0001,
        verbose=1
    )
]

### 3.2 모델 학습
정의된 각 MLP 모델 구성에 대해 다음 과정을 반복합니다:
1. 모델 생성 (해당 `build_xxx_ann` 함수 호출)
2. 모델 아키텍처 확인 (`model.summary()`)
3. 모델 학습 (`model.fit()`): 훈련 데이터(X_train_scaled, y_train)로 학습하고, 검증 데이터(X_val_scaled, y_val)로 성능을 모니터링합니다. 콜백 함수가 적용됩니다.
4. 학습 곡선 시각화: 훈련/검증 손실 변화를 확인합니다.
5. 모델 저장 (선택 사항)
6. 학습된 모델 객체 저장

In [ ]:
# 모델 성능 저장 리스트
model_performances = []

# 각 모델 학습 및 평가
for model_config in ann_models:
    model_name = model_config['name']
    print(f"\n{model_name} 모델 학습 시작...")
    
    # 모델 생성
    model = model_config['build_fn'](X_train.shape[1])
    
    # 모델 아키텍처 출력
    model.summary()
    
    # 모델 학습
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=callbacks,
        verbose=1
    )
    
    # 학습 곡선 시각화
    plt.figure(figsize=(14, 7))
    plt.plot(history.history['loss'], label='훈련 손실')
    plt.plot(history.history['val_loss'], label='검증 손실')
    plt.title(f'{model_name} - 학습 곡선')
    plt.xlabel('에포크')
    plt.ylabel('손실 (MSE)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_DIR}/ann_{model_name}_학습곡선.png')
    plt.show()
    
    # 모델 저장 (선택 사항)
    model.save(f'{DATA_OUTPUT_DIR}/ann_{model_name}.h5')
    print(f"{model_name} 모델이 '{DATA_OUTPUT_DIR}/ann_{model_name}.h5'에 저장되었습니다.")
    
    # 모델 성능 저장
    model_performances.append({
        'name': model_name,
        'model': model
    })

## 4. 모델 평가 및 비교
학습된 모델들의 성능을 평가하고 비교합니다.

### 4.1 모델 평가 함수
스케일링된 예측값을 원래 스케일로 변환하고, 성능 지표(RMSE, MAE, R²)를 계산하며, 예측 결과를 시각화하는 함수(`evaluate_model`)를 정의합니다.

In [ ]:
def evaluate_model(model, X, y, dates=None, model_name='모델'):
    """
    모델 성능 평가 함수
    
    Args:
        model: 학습된 모델
        X: 입력 특성
        y: 타겟 값 (스케일링된)
        dates: 날짜 인덱스 (선택 사항)
        model_name: 모델 이름
        
    Returns:
        mse, rmse, mae, r2, y_pred_orig: 평가 지표와 원래 스케일의 예측 값
    """
    # 예측 수행
    y_pred = model.predict(X)
    
    # 스케일링된 성능 지표
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y, y_pred)
    
    # 원래 스케일로 변환
    y_true_orig = scaler_y.inverse_transform(y.reshape(-1, 1)).flatten()
    y_pred_orig = scaler_y.inverse_transform(y_pred).flatten()
    
    # 원래 스케일의 성능 지표
    orig_mse = mean_squared_error(y_true_orig, y_pred_orig)
    orig_rmse = np.sqrt(orig_mse)
    orig_mae = mean_absolute_error(y_true_orig, y_pred_orig)
    r2 = r2_score(y_true_orig, y_pred_orig)
    
    print(f"\n{model_name} 모델 평가:")
    print(f"MSE: {orig_mse:.4f}")
    print(f"RMSE: {orig_rmse:.4f}")
    print(f"MAE: {orig_mae:.4f}")
    print(f"R² Score: {r2:.4f}")
    
    # 시각화 (날짜 제공된 경우)
    if dates is not None:
        plt.figure(figsize=(14, 7))
        plt.plot(dates, y_true_orig, label='실제값')
        plt.plot(dates, y_pred_orig, label='예측값', alpha=0.7)
        plt.title(f'{model_name} - 예측 결과')
        plt.xlabel('날짜')
        plt.ylabel(TARGET_COL)
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f'{OUTPUT_DIR}/ann_{model_name}_예측.png')
        plt.show()
        
        # 산점도 (실제값 vs 예측값)
        plt.figure(figsize=(10, 7))
        plt.scatter(y_true_orig, y_pred_orig, alpha=0.5)
        plt.plot([y_true_orig.min(), y_true_orig.max()], 
                [y_true_orig.min(), y_true_orig.max()], 
                'r--', lw=2)
        plt.title(f'{model_name} - 실제값 vs 예측값')
        plt.xlabel('실제값')
        plt.ylabel('예측값')
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f'{OUTPUT_DIR}/ann_{model_name}_산점도.png')
        plt.show()
    
    return orig_rmse, orig_mae, r2, y_pred_orig

### 4.2 모델 성능 평가
저장된 각 모델 객체를 사용하여 검증 및 테스트 데이터에 대한 예측을 수행하고, `evaluate_model` 함수를 호출하여 성능 지표를 계산 및 시각화합니다.

In [ ]:
# 각 모델 평가
all_metrics = []

for model_info in model_performances:
    model = model_info['model']
    model_name = model_info['name']
    
    # 검증 데이터 평가
    val_rmse, val_mae, val_r2, val_pred = evaluate_model(
        model, X_val, y_val, val_dates, f"{model_name} (검증)"
    )
    
    # 테스트 데이터 평가
    test_rmse, test_mae, test_r2, test_pred = evaluate_model(
        model, X_test, y_test, test_dates, f"{model_name} (테스트)"
    )
    
    # 성능 저장
    all_metrics.append({
        'model': model_name,
        'val_rmse': val_rmse,
        'val_mae': val_mae,
        'val_r2': val_r2,
        'test_rmse': test_rmse,
        'test_mae': test_mae,
        'test_r2': test_r2
    })

### 4.3 모델 비교
계산된 성능 지표를 바탕으로 여러 MLP 모델 간의 성능을 비교합니다.
- **성능 비교 테이블 및 시각화**: 테스트 성능 지표를 표와 그래프로 나타내어 비교합니다.
- **최고 성능 모델 선택**: 테스트 RMSE가 가장 낮은 모델을 최적 모델로 선정합니다.

In [ ]:
# 성능 비교 테이블
metrics_df = pd.DataFrame(all_metrics)
print("\n모델 성능 비교:")
print(metrics_df)

# 모델 비교 시각화
plt.figure(figsize=(14, 10))

plt.subplot(221)
plt.bar(metrics_df['model'], metrics_df['test_rmse'])
plt.title('테스트 RMSE (낮을수록 좋음)')
plt.ylabel('RMSE')
plt.xticks(rotation=45)

plt.subplot(222)
plt.bar(metrics_df['model'], metrics_df['test_mae'])
plt.title('테스트 MAE (낮을수록 좋음)')
plt.ylabel('MAE')
plt.xticks(rotation=45)

plt.subplot(223)
plt.bar(metrics_df['model'], metrics_df['test_r2'])
plt.title('테스트 R² (높을수록 좋음)')
plt.ylabel('R²')
plt.xticks(rotation=45)

plt.subplot(224)
plt.bar(metrics_df['model'], metrics_df['val_rmse'])
plt.title('검증 RMSE (낮을수록 좋음)')
plt.ylabel('RMSE')
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/ann_모델비교.png')
plt.show()

# 최고 성능 모델 선택
best_model_idx = metrics_df['test_rmse'].idxmin()
best_model_name = metrics_df.loc[best_model_idx, 'model']
best_model = model_performances[best_model_idx]['model']
best_rmse = metrics_df.loc[best_model_idx, 'test_rmse']
best_r2 = metrics_df.loc[best_model_idx, 'test_r2']

print(f"\n최고 성능 모델: {best_model_name}")
print(f"테스트 RMSE: {best_rmse:.4f}")
print(f"테스트 R²: {best_r2:.4f}")

### 4.4 시계열 교차 검증 (Time Series Cross-Validation)
본 분석에서는 기본적인 훈련/검증/테스트 분할 방식을 사용했습니다. 더 견고한 모델 평가를 위해 시계열 교차 검증을 추가로 수행합니다.
시계열 데이터에서는 일반적인 k-fold 교차 검증이 아닌, 시간적 의존성을 고려한 `TimeSeriesSplit`을 사용합니다.

In [ ]:
# 시계열 교차 검증 설정
print("\n시계열 교차 검증 수행...")
tscv = TimeSeriesSplit(n_splits=5)

# 스케일링된 전체 데이터
X_all = np.concatenate([X_train, X_val, X_test])
y_all = np.concatenate([y_train, y_val, y_test])

# 최적 모델 구성 선택
best_model_config = ann_models[best_model_idx]['build_fn']
cv_scores = []

# 교차 검증 시각화 준비
plt.figure(figsize=(14, 7))
for i, (train_idx, test_idx) in enumerate(tscv.split(X_all)):
    # 훈련/테스트 분할
    X_cv_train, X_cv_test = X_all[train_idx], X_all[test_idx]
    y_cv_train, y_cv_test = y_all[train_idx], y_all[test_idx]
    
    # 모델 생성 및 학습
    cv_model = best_model_config(X_train.shape[1])
    cv_model.fit(
        X_cv_train, y_cv_train,
        epochs=EPOCHS // 2,  # 시간 단축을 위해 에포크 감소
        batch_size=BATCH_SIZE,
        verbose=0
    )
    
    # 예측 및 스케일 변환
    y_cv_pred = cv_model.predict(X_cv_test, verbose=0)
    
    # 원래 스케일로 변환
    y_cv_true_orig = scaler_y.inverse_transform(y_cv_test.reshape(-1, 1)).flatten()
    y_cv_pred_orig = scaler_y.inverse_transform(y_cv_pred).flatten()
    
    # 성능 평가
    cv_rmse = np.sqrt(mean_squared_error(y_cv_true_orig, y_cv_pred_orig))
    cv_scores.append(cv_rmse)
    
    # 시각화
    plt.subplot(5, 1, i+1)
    plt.plot(y_cv_true_orig[:40], label='실제값')  # 처음 40개 포인트만 표시
    plt.plot(y_cv_pred_orig[:40], label='예측값', alpha=0.7)
    plt.title(f'Fold {i+1} - RMSE: {cv_rmse:.4f}')
    plt.grid(True, alpha=0.3)
    if i == 0:
        plt.legend()

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/ann_교차검증.png')
plt.show()

# 교차 검증 결과 요약
print(f"\n교차 검증 결과 (5-fold TimeSeriesSplit):")
print(f"평균 RMSE: {np.mean(cv_scores):.4f}")
print(f"표준편차: {np.std(cv_scores):.4f}")
print(f"최소 RMSE: {np.min(cv_scores):.4f}")
print(f"최대 RMSE: {np.max(cv_scores):.4f}")

# 교차 검증 성능 분포 시각화
plt.figure(figsize=(10, 6))
plt.boxplot(cv_scores)
plt.title('교차 검증 RMSE 분포')
plt.ylabel('RMSE')
plt.grid(True, alpha=0.3)
plt.savefig(f'{OUTPUT_DIR}/ann_교차검증_분포.png')
plt.show()

## 5. 결과 해석 및 인사이트 도출
최종 분석 결과와 모델 성능을 요약하고 해석합니다.

### 5.1 미래 예측 시뮬레이션
선택된 최적 MLP 모델을 사용하여 미래 기간(`FORECAST_HORIZON`)에 대한 예측을 수행합니다.
**주의**: 이 예측은 미래 시점의 실제 특성 값을 알 수 없으므로, 마지막 관측 시점의 특성 벡터를 그대로 사용하여 예측하는 **단순화된 방식**입니다. 이는 MLP 모델의 일반적인 미래 예측 방식 중 하나이지만, 정확도에 한계가 있을 수 있습니다.

In [ ]:
# 미래 예측을 위한 예시
print("\n미래 예측 시뮬레이션")
print("참고: 아래 코드는 미래 예측을 위한 단순화된 접근 방식입니다.")
print("      실제 미래의 특성 값(lag, rolling 등)은 알 수 없으므로,")
print("      과거 데이터 기반으로 추정하거나 마지막 값을 사용합니다.")
print("      더 정확한 예측을 위해서는 반복적 예측 또는 미래 특성 모델링이 필요할 수 있습니다.")

# 예측 기간 설정 (파라미터 사용)
# forecast_horizon = 30  # 기존 하드코딩 제거

# 마지막 입력 데이터 가져오기 (스케일링 된 상태)
last_x_scaled = X_test[-1].reshape(1, -1)

# 미래 날짜 생성
last_date = df_features.index[-1]
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=FORECAST_HORIZON)

# 예측 함수 (단순화된 방식: 마지막 특성 반복 사용)
def forecast_simple(model, last_features_scaled, n_steps, scaler_y):
    """ 마지막 특성 벡터를 반복 사용하여 미래 값을 예측합니다. """
    # 마지막 특성 벡터를 예측 기간만큼 복제
    future_features = np.tile(last_features_scaled, (n_steps, 1))
    
    # 예측 수행 (스케일링된 값)
    future_predictions_scaled = model.predict(future_features)
    
    # 원래 스케일로 변환
    future_predictions_original = scaler_y.inverse_transform(future_predictions_scaled).flatten()
    
    return future_predictions_original

# 최적 모델을 사용한 예측 (단순화된 방식)
future_values = forecast_simple(
    best_model, last_x_scaled, FORECAST_HORIZON, scaler_y # 파라미터 사용
)

# 예측 결과 시각화
plt.figure(figsize=(14, 7))
# 과거 데이터 (마지막 90일)
history_days = 90
past_dates = df_features.index[-history_days:]
past_values = df_features[TARGET_COL][-history_days:]

plt.plot(past_dates, past_values, label='과거 데이터', color='blue')
plt.plot(future_dates, future_values, label='미래 예측', color='red', linestyle='--')
plt.axvline(x=last_date, color='green', linestyle='-', label='현재')
plt.title(f'{best_model_name} - {FORECAST_HORIZON}일 미래 예측 시뮬레이션') # 시각화 제목에도 파라미터 적용
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/ann_미래예측.png')
plt.show()

# 예측 결과를 DataFrame으로 변환
forecast_df = pd.DataFrame({
    'date': future_dates,
    f'{TARGET_COL}_forecast': future_values
})

print("\n미래 예측 결과 (처음 10일):")
print(forecast_df.head(10))

# 예측 결과 저장
forecast_df.to_csv(f'{DATA_OUTPUT_DIR}/ann_forecast_results.csv', index=False)
print(f"예측 결과가 '{DATA_OUTPUT_DIR}/ann_forecast_results.csv'에 저장되었습니다.")

### 5.2 결론 및 인사이트
전체 분석 과정과 결과를 요약합니다.
- 최적 모델과 그 성능을 제시합니다.
- 모델 구조(단순/복잡)와 성능 간의 관계를 해석합니다.
- ANN 모델의 특성 중요도 분석의 어려움과 대안을 언급합니다.
- 입력 윈도우 크기의 영향을 설명합니다.
- 향후 개선 방향(하이퍼파라미터 튜닝, 특성 조정, 다른 모델 비교 등)을 제안합니다.

In [ ]:
print("\n" + "="*50)
print("결론 및 인사이트")
print("="*50)

# 모델 성능 요약
print(f"1. 최적의 모델: {best_model_name}")
print(f"2. 테스트 RMSE: {best_rmse:.4f}")
print(f"3. 테스트 R²: {best_r2:.4f}")

# 모델 특성
if best_model_name == 'SimpleANN':
    print("\n단순 구조 모델이 최적 성능을 보였습니다:")
    print("- 데이터의 패턴이 비교적 단순하거나 모델의 과적합이 발생했을 수 있습니다.")
    print("- 단순 모델은 일반화 능력이 더 좋은 경우가 많습니다.")
elif best_model_name == 'DeepANN':
    print("\n중간 깊이의 모델이 최적 성능을 보였습니다:")
    print("- 모델 복잡성과 일반화 능력 사이의 균형이 잘 맞았습니다.")
    print("- 다층 구조가 데이터의 비선형 패턴을 더 잘 포착했습니다.")
else:  # ComplexANN
    print("\n복잡한 모델이 최적 성능을 보였습니다:")
    print("- 데이터가 복잡한 패턴을 가지고 있어 더 깊은 모델이 필요했습니다.")
    print("- 드롭아웃이 과적합 방지에 효과적이었습니다.")

# 특성 중요도 분석 (추후 구현)
print("\n특성 중요도는 ANN 모델에서 직접 계산하기 어렵습니다.")
print("별도의 방법론(예: 순열 중요도, SHAP 값)을 통해 분석할 수 있습니다.")

# 윈도우 크기 분석
print(f"\n입력 윈도우 크기({WINDOW_SIZE})가 예측 성능에 영향을 미쳤습니다:")
print(f"- 더 긴 윈도우는 장기 패턴을 포착할 수 있지만 훈련 데이터가 줄어듭니다.")
print(f"- 더 짧은 윈도우는 최근 패턴에 집중하지만 장기 패턴을 놓칠 수 있습니다.")

# 향후 개선 방향
print("\n향후 개선 방향:")
print("1. 하이퍼파라미터 최적화를 통한 성능 향상")
print("2. 윈도우 크기 및 특성 엔지니어링 조정")
print("3. LSTM 등 순환 신경망과의 성능 비교")
print("4. 앙상블 방법을 통한 여러 모델 결합")
print("5. 미래 예측을 위한 더 정교한 접근법 개발")

print("\n인공신경망 기반 시계열 분석 완료!") 